##  B7. Cloud mask production

#### 0. Load required libraries,  area dependant constants and utility functions:

In [2]:
from os import listdir
from os.path import isfile, join
import numpy as np
import seaborn as sns

# EOLearn libraries:
from eolearn.core import EOTask, EOPatch, LinearWorkflow, LoadTask, SaveTask, FeatureType, EOExecutor
from eolearn.core import OverwritePermission

# Add to python path parent dictionary (to have acces to the data and util catalogue)
import sys
sys.path.append("../../")

# load site dependant constants (HERE YOU CAN CHOOSE DIFFERENT LOCATION)
from aoi_sites import upe_promice_area as site

# load utility functions
from utils import io_functions as io_utils
from utils import plot_functions as plot_utils

#### 1. Set up and execute EOWorkflow

In [7]:
# defining eo_task for water feature extraction
import numpy.ma as ma
class StaticThreshioldingForClouds(EOTask):   
    """
    The tasks applies a static threshold to produce a data mask:
    MASK = 1 if > x else 0
    """
    def __init__(self, data_name, band_no, feature_name, threshold):
        self.feature_name = feature_name
        self.data_name = data_name
        self.band_no = band_no
        self.threshold = threshold
        
    def execute(self, eopatch):
        band_a = eopatch.data[self.feature_name][..., self.band_no]
        mask = ma.masked_where(self.threshold <= band_a , band_a)
        #any will return True if there's any truth value in the iterable.
        #print(mask.mask)
        #print(mask.mask.any())
        #print(np.isnan(mask.mask).any())
        if mask.mask.any() == False:
            new_mask = np.zeros(mask.shape, dtype=bool)
            mask.mask = new_mask
            print('Created new mask of zeros, bc there was no water feature detected')
            
        eopatch.add_feature(FeatureType.MASK, self.data_name, mask.mask[..., np.newaxis])
        return eopatch

#### 2. Set up and execute EOWorkflow

In [11]:
# creating different eo tasks  producing Clouds mask - for sanity check
static_threshol_090_eotask = StaticThreshioldingForClouds('CLOUD_MASK_SWIR-1_090', 6, 'LANDSAT_RAW_BANDS', 0.090)
static_threshol_100_eotask = StaticThreshioldingForClouds('CLOUD_MASK_SWIR-1_100', 6, 'LANDSAT_RAW_BANDS', 0.100)
static_threshol_150_eotask = StaticThreshioldingForClouds('CLOUD_MASK_SWIR-1_150', 6, 'LANDSAT_RAW_BANDS', 0.150)
static_threshol_200_eotask = StaticThreshioldingForClouds('CLOUD_MASK_SWIR-1_200', 6, 'LANDSAT_RAW_BANDS', 0.200)
static_threshol_300_eotask = StaticThreshioldingForClouds('CLOUD_MASK_SWIR-1_300', 6, 'LANDSAT_RAW_BANDS', 0.300)

# application of all eotasks for all eopatches in the a eoworkflow
for date_range_landsat in site.DATE_RANGES_LANDSAT_8 :
    print('Reading EOPatches from:', date_range_landsat)
    eopatches_filepath = io_utils.get_eopatches_dir(site.DATA_PRODUCTS[1], site.SITE_NAME, site.AVAILABLE_CRSs[0], date_range_landsat)
    # TASK TO LOAD AND SAVE EXISTING EOPATCHES HAS TO BE CREATED YEARLY (different directories)
    load_eotask = LoadTask(eopatches_filepath)
    save_eotask = SaveTask(eopatches_filepath, overwrite_permission=OverwritePermission.OVERWRITE_PATCH, compress_level=1)
    
    # Define the workflow
    workflow = LinearWorkflow(
    load_eotask,
    static_threshol_090_eotask,
    static_threshol_100_eotask,
    static_threshol_150_eotask,
    static_threshol_200_eotask,
    static_threshol_300_eotask,
    save_eotask
    )
                      
    list_of_available_patches = io_utils.get_list_of_eopatches(eopatches_filepath)
    execution_args = []
    for eopatch_name in list_of_available_patches:
        execution_args.append({
        load_eotask: {'eopatch_folder': eopatch_name},
        save_eotask: {'eopatch_folder': eopatch_name}
        })
    
    #print(execution_args)                            
    executor = EOExecutor(workflow, execution_args, save_logs=True)
    executor.run(workers=1, multiprocess=False)

    executor.make_report()

Reading EOPatches from: ('2013-05-01', '2013-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2014-05-01', '2014-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created ne

Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created ne

Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2017-05-01', '2017-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created ne

Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

Reading EOPatches from: ('2019-05-01', '2019-10-31')


Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected
Created new mask of zeros, bc there was no water feature detected

